In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SimpleITK
!pip install pyradiomics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117

**Import necessary packages**

In [ ]:
import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor

**Initialize the extractor with the YAML configuration file**

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor('/content/drive/MyDrive/FYP/FeatureExtraction/params.yaml')

**Paths to image and mask folders**

In [ ]:
image_folder = "/content/drive/MyDrive/FYP/segmentation/Covid"
mask_folder = "/content/drive/MyDrive/FYP/Masks"

**Prepare the output CSV file**

In [ ]:
output_csv = "/content/drive/MyDrive/FYP/extracted_features.csv"
feature_data = []

**Get all image and mask files (assuming they are paired and ordered in the folders)**

In [ ]:
image_files = sorted(os.listdir(image_folder))
mask_files = sorted(os.listdir(mask_folder))

**Loop through each image and corresponding mask**

In [ ]:
for img_file, mask_file in zip(image_files, mask_files):
    image_path = os.path.join(image_folder, img_file)
    mask_path = os.path.join(mask_folder, mask_file)

    # Read the image and mask
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)

    # Convert images to grayscale if necessary
    if image.GetNumberOfComponentsPerPixel() > 1:
        image = sitk.VectorIndexSelectionCast(image, 0)  # Select one channel, e.g., Red channel

    if mask.GetNumberOfComponentsPerPixel() > 1:
        mask = sitk.VectorIndexSelectionCast(mask, 0)  # Ensure mask is single-channel as well

    # Ensure images are of type UInt8 or UInt16, compatible with radiomics processing
    image = sitk.Cast(image, sitk.sitkUInt8)
    mask = sitk.Cast(mask, sitk.sitkUInt8)

    # Check if the image and mask sizes match
    if image.GetSize() != mask.GetSize():
        print(f"Resizing mask for {img_file} to match image dimensions.")
        mask = sitk.Resample(mask, referenceImage=image, transform=sitk.Transform(),
                             interpolator=sitk.sitkNearestNeighbor, defaultPixelValue=0, outputPixelType=sitk.sitkUInt8)

    # Check if the label 255 exists in the mask
    mask_stats = sitk.LabelStatisticsImageFilter()
    mask_stats.Execute(image, mask)

    if 255 in mask_stats.GetLabels():  # Proceed only if label 255 is present
        # Extract only the specified features
        features = extractor.execute(image, mask)

        # Store features in a dictionary, with image filename as identifier
        feature_row = {"Image": img_file}
        feature_row.update({feature_name: feature_value for feature_name, feature_value in features.items()})

    else:
        # If label 255 is missing, add an empty row for this image
        print(f"No segmentation found in {img_file}. Adding empty row.")
        feature_row = {"Image": img_file}

    # Append the row to the data list
    feature_data.append(feature_row)

**Convert the list of dictionaries to a DataFrame**

In [ ]:
df = pd.DataFrame(feature_data)

**Save the DataFrame to CSV**

In [ ]:
df.to_csv(output_csv, index=False)
print(f"Feature extraction complete. Results saved to {output_csv}")